# Visualize Performance of Cell Health Prediction Models

**Gregory Way, 2019**

For regression classification problem. 

In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(ggrepel))
suppressPackageStartupMessages(library(ggpmisc))

## Load Data

In [2]:
results_dir <- "results"
regression_file <- file.path(results_dir, "full_cell_health_regression_results.tsv.gz")
regression_metrics_df <- readr::read_tsv(regression_file, col_types = readr::cols())

coef_file <- file.path(results_dir, "full_cell_health_coefficients.tsv.gz")
full_coef_df <- readr::read_tsv(coef_file, col_types = readr::cols()) %>%
    dplyr::filter(y_transform %in% c("raw", "zero-one"))

y_file <- file.path(results_dir, "full_cell_health_y_labels.tsv.gz")
y_df <- readr::read_tsv(y_file, col_types = readr::cols()) %>%
    dplyr::filter(y_transform %in% c("raw", "zero-one"))

## Process y Matrix for Plotting

In [3]:
# Combine data for downstream processing
y_binary_subset_true_df <- y_df %>%
    dplyr::filter(y_transform %in% c("raw", "zero-one"),
                  y_type == "y_true")

y_binary_subset_pred_df <- y_df %>%
    dplyr::filter(y_transform %in% c("raw", "zero-one"),
                  y_type == "y_pred")

# Process data for plotting
y_plot_df <- y_binary_subset_true_df %>%
    dplyr::inner_join(y_binary_subset_pred_df,
                      by = c("Metadata_profile_id",
                             "target",
                             "data_type",
                             "shuffle",
                             "y_transform"),
                      suffix = c("_true", "_pred"))

y_plot_df$data_type <- dplyr::recode(y_plot_df$data_type,
                                     "train" = "Train",
                                     "test" = "Test")

print(dim(y_plot_df))
head(y_plot_df, 3)

Warning message:
“`chr_along()` is deprecated as of rlang 0.2.0.
This warning is displayed once per session.”

[1] 99720     9


Metadata_profile_id,recode_target_value_true,target,data_type,shuffle,y_transform,y_type_true,recode_target_value_pred,y_type_pred
<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
profile_346,-0.7839227,cc_all_high_n_spots_h2ax_mean,Train,shuffle_true,raw,y_true,0.4657851,y_pred
profile_185,4.8193411,cc_all_high_n_spots_h2ax_mean,Train,shuffle_true,raw,y_true,0.4657851,y_pred
profile_3,-0.1389207,cc_all_high_n_spots_h2ax_mean,Train,shuffle_true,raw,y_true,0.4657851,y_pred


## Process Regression Metrics for Plotting

In [4]:
regression_metrics_df$data_fit <- dplyr::recode(regression_metrics_df$data_fit,
                                                "train" = "Train",
                                                "test" = "Test")

regression_metrics_df <- regression_metrics_df %>%
    dplyr::rename(data_type = data_fit)

regression_metrics_df$mse <- round(regression_metrics_df$mse, 2)

print(dim(regression_metrics_df))
head(regression_metrics_df, 3)

[1] 1120    6


mse,metric,target,data_type,shuffle,y_transform
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1.85,mse,cc_all_high_n_spots_h2ax_mean,Train,shuffle_true,raw
0.92,mse,cc_all_high_n_spots_h2ax_mean,Test,shuffle_true,raw
0.00,r_two,cc_all_high_n_spots_h2ax_mean,Train,shuffle_true,raw


## Plot performance metrics for each target variable

In [5]:
label_thresh_value = 0.90

pdf_file <- file.path("figures", "all_regression_performance_metrics.pdf")
pdf(pdf_file, width = 6, height = 8, onefile = TRUE)

for (target in unique(y_plot_df$target)) {
    # Subset all dataframes
    y_subset_df <- y_plot_df %>% dplyr::filter(target == !!target)
    coef_subset_df <- full_coef_df %>% dplyr::filter(target == !!target)
    metrics_subset_df <- regression_metrics_df %>% dplyr::filter(target == !!target)
    
    for (y_transform in unique(y_subset_df$y_transform)) {
        y_subset_transform_df <- y_subset_df %>%
            dplyr::filter(y_transform == !!y_transform)
       
        coef_subset_transform_df <- coef_subset_df %>%
            dplyr::filter(y_transform == !!y_transform,
                          shuffle == "shuffle_false") %>%
            dplyr::mutate(weight_rank = row_number(weight))
        
        metrics_subset_transform_df <- metrics_subset_df %>%
            dplyr::filter(y_transform == !!y_transform)
        
        pred_scatter_gg <-
           ggplot(y_subset_transform_df,
                  aes(x = recode_target_value_true, y = recode_target_value_pred)) +
                geom_point(size = 0.5, alpha = 0.7) +
                facet_grid(data_type~shuffle) +
                theme_bw() +
                xlab("True Values") +
                ylab("Predicted Values") +
                geom_smooth(method='lm', formula=y~x) +
                theme(strip.text = element_text(size = 10),
                      strip.background = element_rect(colour = "black",
                                                      fill = "#fdfff4"))
        
        # Setup labeling thresholds
        non_zero_coef <- coef_subset_transform_df$abs_weight[coef_subset_transform_df$abs_weight > 0]
        label_thresh <- quantile(non_zero_coef, label_thresh_value)
        label_logic <- coef_subset_transform_df$abs_weight > label_thresh

        coef_gg <-
            ggplot(coef_subset_transform_df,
                   aes(x = weight_rank,
                       y = weight)) +
                geom_point(size = 0.2,
                           alpha = 0.6) +
                xlab("Weight Rank") +
                ylab("Weight") +
                geom_text_repel(data = subset(coef_subset_transform_df, label_logic),
                                arrow = arrow(length = unit(0.01, "npc")),
                                box.padding = 0.4,
                                point.padding = 0.1,
                                segment.size = 0.5,
                                segment.alpha = 0.6,
                                size = 1.5,
                                fontface = "italic",
                                aes(label = feature,
                                    x = weight_rank,
                                    y = weight)) +
                theme_bw()
        
         # Build table for plotting performance metrics
        mse_df <- metrics_subset_transform_df %>%
            dplyr::filter(metric == "mse") %>%
            dplyr::select(-metric)
        r2_df <- metrics_subset_transform_df %>%
            dplyr::filter(metric == "r_two") %>%
            dplyr::rename(r2 = mse) %>%
            dplyr::select(-metric)

        metric_table_df <- r2_df %>%
            dplyr::inner_join(mse_df, by=c("target", "data_type", "shuffle", "y_transform")) %>%
            dplyr::select(data_type, shuffle, y_transform, r2, mse) %>%
            dplyr::rename(fit = data_type, transform = y_transform) %>%
            dplyr::arrange(shuffle)

        metric_table_df$shuffle <- dplyr::recode(metric_table_df$shuffle,
                                              shuffle_true = "True",
                                              shuffle_false = "False")

        # Plot all performance metrics together with cowplot
        table_theme <- gridExtra::ttheme_default(
            core = list(fg_params=list(cex = 0.7)),
            colhead = list(fg_params=list(cex = 0.8))
        )

        table_gg <- gridExtra::tableGrob(metric_table_df,
                                         theme = table_theme,
                                         rows = NULL)
        
        bottom_row_gg <- cowplot::plot_grid(
            table_gg,
            coef_gg,
            rel_widths = c(0.8, 1),
            nrow = 1
        )
        regression_perf_gg <- cowplot::plot_grid(
            pred_scatter_gg,
            bottom_row_gg,
            rel_heights = c(1, 0.8),
            nrow = 2
        )
        
        target_title <- cowplot::ggdraw() + 
          cowplot::draw_label(
            paste("Performance:", target, "\nTransform:", y_transform),
            fontface = 'bold',
            x = 0,
            hjust = 0
          ) +
          theme(
            plot.margin = margin(0, 0, 0, 7)
          )

        regression_perf_gg <- cowplot::plot_grid(
            target_title,
            regression_perf_gg,
            ncol = 1,
            rel_heights = c(0.1, 1)
        )
        
        # Save figure
        cowplot_file <- file.path("figures",
                                  "target_performance",
                                  "regression",
                                  paste0(target, "_", y_transform, "_performance.png"))

        cowplot::save_plot(filename = cowplot_file,
                           plot = regression_perf_gg,
                           base_height = 6,
                           base_width = 8)

        print(regression_perf_gg)
    }
}

dev.off()

Warning message:
“The `printer` argument is deprecated as of rlang 0.3.0.
This warning is displayed once per session.”

pdf 
  2